### Read in the Libraries and Dataset

In [1]:
# Import libraries for our task
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

import dvc.api
import numpy as np
import pandas as pd
import seaborn as sns 
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots

In [2]:
# Import system libraries and our Scripts

import os
import sys
import warnings
warnings.filterwarnings('ignore')
sys.path.append(os.path.abspath(os.path.join('../script')))
from train_classifiers import TrainingClassifier 

In [3]:
# Read AdSmart_AB_testing_data 

cleaned_data  = pd.read_csv('../data/clean_data.csv')

In [4]:
# The types and shape of data features in the dataset

cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1243 entries, 0 to 1242
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1243 non-null   int64 
 1   auction_id   1243 non-null   object
 2   experiment   1243 non-null   object
 3   date         1243 non-null   object
 4   hour         1243 non-null   int64 
 5   device_make  1243 non-null   object
 6   platform_os  1243 non-null   int64 
 7   browser      1243 non-null   object
 8   aware        1243 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 87.5+ KB


In [10]:
def get_data(tag, path='data/AdSmart_AB_testing_data.csv', repo='https://github.com/bkget/SmartAd-Performance-Analysis'):
    rev = tag
    data_url = dvc.api.get_url(path=path, repo=repo, rev=rev)
    df = pd.read_csv(data_url)
    return df

In [ ]:
read = get_data('v1')

##### Label Encoding the  Data

In [1]:
def encode_labels(df):
    date_encoder = preprocessing.LabelEncoder()
    device_encoder = preprocessing.LabelEncoder()
    browser_encoder = preprocessing.LabelEncoder()
    experiment_encoder = preprocessing.LabelEncoder()
    aware_encoder = preprocessing.LabelEncoder()
    
    df['date'] = date_encoder.fit_transform(df['date'])
    df['device_make'] = device_encoder.fit_transform(df['device_make'])
    df['browser'] = browser_encoder.fit_transform(df['browser'])
    df['experiment'] = experiment_encoder.fit_transform(df['experiment'])
    df['browser'] = aware_encoder.fit_transform(df['browser'])
    df['aware'] = aware_encoder.fit_transform(df['aware'])
    
    return df    

##### Spliting the data by browser and platform_os columns

In [55]:
def feature_data(df):
    
    browser_feature_df = df[["experiment", "hour", "date", 'device_make', 'browser', 'aware']] 
    platform_feature_df = df[["experiment", "hour", "date", 'device_make', 'platform_os', 'aware']] 

    return browser_feature_df, platform_feature_df

In [56]:
def save_encoded_df(df):
    
    broweser_df, platfrom_df = feature_data(df)
    broweser_df.save_csv("../Data/browser_featured_data.csv")
    platfrom_df.save_csv(platfrom_df, "../data/platform_featured_data.csv") 

In [57]:
cleaned_data = encode_labels(cleaned_data)
cleaned_browser_df, cleaned_platform_df = feature_data(cleaned_data)

In [58]:
cleaned_browser_df.head()

,experiment,hour,date,device_make,browser,aware
0,1,2,2,4,2,0
1,1,16,1,13,1,1
2,1,8,3,13,1,0
3,0,4,5,43,4,1
4,0,15,0,13,1,0


In [59]:
Y_col = 'aware'
X_cols = cleaned_browser_df.loc[:, cleaned_browser_df.columns != Y_col].columns

X_train,X_test,y_train,y_test=train_test_split(cleaned_browser_df[X_cols], cleaned_browser_df[Y_col],\
                                                test_size=0.1, random_state=42)